In [1]:
!pip install -q transformers accelerate sentencepiece


In [2]:
from google.colab import files
uploaded = files.upload()


Saving core-0.1.1.py to core-0.1.1.py
Saving core-0.2.2.py to core-0.2.2.py
Saving core-0.2.4.py to core-0.2.4.py


In [4]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "Qwen/Qwen2.5-Coder-7B-Instruct"
MODEL_NAME = "Qwen-2.5-Coder"

SYSTEM_PROMPT = """
Atue como um Engenheiro de Software Sênior especialista em Python e Engenharia de Software.
Analise o código procurando por Code Smells conforme o Refactoring Guru,
com foco em: Long Method, Magic Numbers e Cognitive Complexity.

Para cada code smell identificado, forneça:
- Localização
- Evidência
- Justificativa técnica
- Sugestão de refatoração
"""

device = "cuda"
dtype = torch.float16


def carregar_modelo():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=dtype,
        low_cpu_mem_usage=True
    ).eval()

    return tokenizer, model


def analisar_codigo(codigo, tokenizer, model):
    MAX_CHARS = 2500

    codigo = codigo[:MAX_CHARS]

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Analise o código:\n\n{codigo}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(device)

    attention_mask = input_ids.ne(tokenizer.pad_token_id)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=300,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta = tokenizer.decode(
        output[0][input_ids.shape[1]:],
        skip_special_tokens=True
    )

    return resposta


In [5]:
tokenizer, model = carregar_modelo()

resultados = {}

for nome_arquivo in uploaded.keys():
    print(f"\n📂 Analisando: {nome_arquivo}")

    with open(nome_arquivo, "r", encoding="utf-8") as f:
        codigo = f.read()

    resultado = analisar_codigo(codigo, tokenizer, model)
    resultados[nome_arquivo] = resultado

    print("\n--- RESULTADO ---\n")
    print(resultado)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



📂 Analisando: core-0.1.1.py

--- RESULTADO ---

### Análise do Código

O código apresentado é uma classe `Chat` que parece ser responsável pela gestão de modelos de linguagem e outras funcionalidades relacionadas. Vamos analisar o código em busca de Code Smells, com foco em Long Method, Magic Numbers e Cognitive Complexity.

#### 1. Long Method

**Localização:** `download_models`

**Evidência:**
```python
def download_models(
    self,
    source: Literal["huggingface", "local", "custom"] = "local",
    force_redownload=False,
    custom_path: Optional[torch.serialization.FILE_LIKE] = None,
) -> Optional[str]:
    if source == "local":
        download_path = os.getcwd()
        if (
            not check_all_assets(Path(download_path), self.sha256_map, update=True)
            or force_redownload
        ):
            with tempfile.TemporaryDirectory() as tmp:
                download_all_assets(tmpdir=tmp)
            if not check_all_assets(
                Path(download_path), se

In [6]:
from google.colab import files

for nome, texto in resultados.items():
    saida = f"resultado_{nome.replace('.py','')}.txt"

    with open(saida, "w", encoding="utf-8") as f:
        f.write(texto)

    files.download(saida)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>